In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import os
from scipy.misc import imread,imresize
from random import shuffle
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from tqdm import tqdm
import mobilenet_v2
import warnings
warnings.filterwarnings('ignore')
tf.__version__

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.5.0'

In [2]:
checkpoint_name = 'mobilenet_v2_1.0_224'
checkpoint = checkpoint_name + '.ckpt'

In [3]:
batch_size = 32
learning_rate = 1e-3
data_location = 'Crop/'

In [4]:
img_lists = os.listdir(data_location)
shuffle(img_lists)
img_labels = [i.split('--')[0] for i in img_lists]
img_Y = LabelEncoder().fit_transform(img_labels)
img_lists = [data_location+i for i in img_lists]
label_emotions = ['anger','contempt','disgust','fear','happy','sadness','surprise']

In [5]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(img_lists):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [11784 11785 11786 ... 35349 35350 35351] TEST: [    0     1     2 ... 11781 11782 11783]
TRAIN: [    0     1     2 ... 35349 35350 35351] TEST: [11784 11785 11786 ... 23565 23566 23567]
TRAIN: [    0     1     2 ... 23565 23566 23567] TEST: [23568 23569 23570 ... 35349 35350 35351]


In [6]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
X = tf.placeholder(tf.float32,[None,224,224,1])
Y = tf.placeholder(tf.int32, [None])
images = tf.image.grayscale_to_rgb(X)
images = tf.map_fn(lambda image: tf.image.per_image_standardization(image), images)
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope(is_training=True)):
    logits, endpoints = mobilenet_v2.mobilenet(images,num_classes=len(label_emotions))
emotion_cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
cost = tf.reduce_mean(emotion_cross_entropy)
emotion_accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logits, Y, 1), tf.float32))
global_step = tf.Variable(0, name="global_step", trainable=False)

tf.summary.scalar("total_loss", cost)
tf.summary.scalar("emotion_accuracy", emotion_accuracy)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost,global_step=global_step)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'MobilenetV2')
var_lists_without_logits = [i for i in var_lists if i.name.find('Conv2d_1c') < 0]
saver = tf.train.Saver(var_list = var_lists_without_logits)
saver.restore(sess, checkpoint)
saver = tf.train.Saver(tf.global_variables())
# test save
saver.save(sess, "huseinhouse/emotion-checkpoint-mobilenet.ckpt")

INFO:tensorflow:Restoring parameters from mobilenet_v2_1.0_224.ckpt


'huseinhouse/emotion-checkpoint-mobilenet.ckpt'

In [7]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

Open new terminal in current directory,
```bash
tensorboard --logdir=./logs
```

In [8]:
for no, indexes in enumerate(kf.split(img_lists)):
    print('--------------- Start cross validation %d ---------------'%(no))
    saver.restore(sess, "huseinhouse/emotion-checkpoint-mobilenet.ckpt")
    
    train_X = np.array(img_lists)[indexes[0]]
    train_Y = img_Y[indexes[0]]
    
    test_X = np.array(img_lists)[indexes[1]]
    test_Y = img_Y[indexes[1]]
    
    EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
    while True:
        if CURRENT_CHECKPOINT == EARLY_STOPPING:
            print('break epoch:%d\n'%(EPOCH))
            break
        
        train_loss, train_acc, test_loss, test_acc = 0, 0, 0, 0
        for k in tqdm(range(0, (len(train_X) // batch_size) * batch_size, batch_size),
                      desc='train minibatch loop'):
            batch_x = np.zeros((batch_size, 224,224,1))
            for n in range(batch_size):
                img = imresize(imread(train_X[k+n]), (224,224))
                batch_x[n,:,:,0] = img
            summary,loss, acc, _ = sess.run([merged,cost,emotion_accuracy,optimizer],
                                    feed_dict={X:batch_x,Y:train_Y[k:k+batch_size]})
            train_loss += loss
            train_acc += acc
            writer.add_summary(summary, global_step=sess.run(global_step))
        
        for k in tqdm(range(0, (len(test_X) // batch_size) * batch_size, batch_size),
                      desc='test minibatch loop'):
            batch_x = np.zeros((batch_size, 224,224,1))
            for n in range(batch_size):
                img = imresize(imread(test_X[k+n]), (224,224))
                batch_x[n,:,:,0] = img
            loss, acc = sess.run([cost,emotion_accuracy], 
                                 feed_dict={X:batch_x,Y:test_Y[k:k+batch_size]})
            test_loss += loss
            test_acc += acc
        
        train_loss /= (len(train_X) // batch_size)
        train_acc /= (len(train_X) // batch_size)
        test_loss /= (len(test_X) // batch_size)
        test_acc /= (len(test_X) // batch_size)
        
        if test_acc > CURRENT_ACC:
            print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
            CURRENT_ACC = test_acc
            CURRENT_CHECKPOINT = 0
            saver.save(sess, "huseinhouse/emotion-checkpoint-mobilenet.ckpt")
        else:
            CURRENT_CHECKPOINT += 1
            
        print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
        EPOCH += 1
    print('--------------- End cross validation %d -----------------\n\n'%(no))

--------------- Start cross validation 0 ---------------
INFO:tensorflow:Restoring parameters from huseinhouse/emotion-checkpoint-mobilenet.ckpt


test minibatch loop: 100%|██████████| 368/368 [01:11<00:00,  5.11it/s]


epoch: 0, pass acc: 0.000000, current acc: 0.777938


train minibatch loop:   0%|          | 0/736 [00:00<?, ?it/s]

epoch: 0, training loss: 0.834706, training acc: 0.705206, valid loss: 0.624796, valid acc: 0.777938



...

epoch: 3, training loss: 0.084944, training acc: 0.970321, valid loss: 0.148349, valid acc: 0.950153



train minibatch loop:   0%|          | 0/736 [00:00<?, ?it/s]

epoch: 4, training loss: 0.081592, training acc: 0.970746, valid loss: 0.142782, valid acc: 0.951257



test minibatch loop: 100%|██████████| 368/368 [01:11<00:00,  5.11it/s]

epoch: 5, training loss: 0.062490, training acc: 0.978218, valid loss: 0.198886, valid acc: 0.941151

break epoch:6

--------------- End cross validation 2 -----------------


